In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime

In [2]:
df = pd.read_csv('/kaggle/input/export-data/export_26550776.csv') 
df = pd.DataFrame(df, columns=['CREATED_TIMESTAMP', 'METRIC_VALUE_FLOAT'])


df['time'] = df['CREATED_TIMESTAMP']

df['time'] = pd.to_datetime(pd.Series(df['time']), format="%d-%b-%y %I.%M.%S %p")
df = df.rename(columns={"METRIC_VALUE_FLOAT": "cnt"})

df.index = df.time
df.head()

,CREATED_TIMESTAMP,cnt,time
time,,,
2019-04-29 23:50:00,29-APR-19 11.50.00 PM,0.93,2019-04-29 23:50:00
2019-04-29 23:55:00,29-APR-19 11.55.00 PM,1.50,2019-04-29 23:55:00
2019-04-29 23:30:00,29-APR-19 11.30.00 PM,1.43,2019-04-29 23:30:00
2019-04-29 23:45:00,29-APR-19 11.45.00 PM,1.74,2019-04-29 23:45:00
2019-04-29 23:00:00,29-APR-19 11.00.00 PM,1.97,2019-04-29 23:00:00


In [3]:
df['weekday'] = df.time.dt.weekday
df.head()

,CREATED_TIMESTAMP,cnt,time,weekday
time,,,,
2019-04-29 23:50:00,29-APR-19 11.50.00 PM,0.93,2019-04-29 23:50:00,0
2019-04-29 23:55:00,29-APR-19 11.55.00 PM,1.50,2019-04-29 23:55:00,0
2019-04-29 23:30:00,29-APR-19 11.30.00 PM,1.43,2019-04-29 23:30:00,0
2019-04-29 23:45:00,29-APR-19 11.45.00 PM,1.74,2019-04-29 23:45:00,0
2019-04-29 23:00:00,29-APR-19 11.00.00 PM,1.97,2019-04-29 23:00:00,0


In [4]:
df['hour'] = df.time.dt.hour
df.head()

,CREATED_TIMESTAMP,cnt,time,weekday,hour
time,,,,,
2019-04-29 23:50:00,29-APR-19 11.50.00 PM,0.93,2019-04-29 23:50:00,0,23
2019-04-29 23:55:00,29-APR-19 11.55.00 PM,1.50,2019-04-29 23:55:00,0,23
2019-04-29 23:30:00,29-APR-19 11.30.00 PM,1.43,2019-04-29 23:30:00,0,23
2019-04-29 23:45:00,29-APR-19 11.45.00 PM,1.74,2019-04-29 23:45:00,0,23
2019-04-29 23:00:00,29-APR-19 11.00.00 PM,1.97,2019-04-29 23:00:00,0,23


In [5]:
def day_splitter(df):
    l = []
    for day in range(7):
        d = df[df['weekday']==day]
        l.append(d)
    return l

dfs = day_splitter(df)
len(dfs)


print(len(dfs[0]))
dfs[0].head()

1545


,CREATED_TIMESTAMP,cnt,time,weekday,hour
time,,,,,
2019-04-29 23:50:00,29-APR-19 11.50.00 PM,0.93,2019-04-29 23:50:00,0,23
2019-04-29 23:55:00,29-APR-19 11.55.00 PM,1.50,2019-04-29 23:55:00,0,23
2019-04-29 23:30:00,29-APR-19 11.30.00 PM,1.43,2019-04-29 23:30:00,0,23
2019-04-29 23:45:00,29-APR-19 11.45.00 PM,1.74,2019-04-29 23:45:00,0,23
2019-04-29 23:00:00,29-APR-19 11.00.00 PM,1.97,2019-04-29 23:00:00,0,23


In [6]:
def hour_splitter(df):
    l = []
    for hour in range(24):
        try:
            d = df[df['hour']==hour]
            l.append(d)
        except KeyError: pass
    return l

In [7]:
def get_dfs(dfs):
    l = []
    for df in dfs:
        l.extend(hour_splitter(df))
    return l

In [8]:
def anomaly_detector(df):
    m, s = df.mean()[0], df.std()[0]
    cutoff = s*3
    lower, upper = m - cutoff, m + cutoff
    anomalies = df['cnt'].apply(lambda x: x<lower or x>upper)
    idx = anomalies.values.reshape(-1)
    return idx

final_dfs = get_dfs(dfs)

len(final_dfs)

anomaly = []

In [9]:
for f_df in final_dfs:
    l = anomaly_detector(f_df)
    for i in range(len(l)):
        if l[i]: 
            d_time = f_df.iloc[i].time
            anomaly.append(d_time)
            
            
df['SerialNo'] = range(len(df))
df.head()

/tmp/ipykernel_32/421368774.py:2: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  m, s = df.mean()[0], df.std()[0]
/tmp/ipykernel_32/421368774.py:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  m, s = df.mean()[0], df.std()[0]
/tmp/ipykernel_32/421368774.py:2: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  m, s = df.mean()[0], df.std()[0]
/tmp/ipykernel_32/421368774.py:2: FutureWarning: DataFrame.mean and DataFrame.median with numeric_

,CREATED_TIMESTAMP,cnt,time,weekday,hour,SerialNo
time,,,,,,
2019-04-29 23:50:00,29-APR-19 11.50.00 PM,0.93,2019-04-29 23:50:00,0,23,0
2019-04-29 23:55:00,29-APR-19 11.55.00 PM,1.50,2019-04-29 23:55:00,0,23,1
2019-04-29 23:30:00,29-APR-19 11.30.00 PM,1.43,2019-04-29 23:30:00,0,23,2
2019-04-29 23:45:00,29-APR-19 11.45.00 PM,1.74,2019-04-29 23:45:00,0,23,3
2019-04-29 23:00:00,29-APR-19 11.00.00 PM,1.97,2019-04-29 23:00:00,0,23,4


In [14]:
a = []
for l in df.time.tolist():
    if l in anomaly:
        a.append(1)
    else:
        a.append(0)

df['anomaly'] = a

no_anomaly_df = df[df.anomaly == 0].reset_index().sort_values('time')
anomaly_df = df[df.anomaly == 1].reset_index().sort_values('time')

fig, ax = plt.subplots(figsize=(20, 10))
ax.plot(no_anomaly_df['SerialNo'], no_anomaly_df['cnt'], color='blue', label='Normal')
ax.scatter(anomaly_df['SerialNo'], anomaly_df['cnt'], color='red', label='Anomaly')
plt.title('Anomalies')
plt.xlabel('Date Time integer')
plt.ylabel('Count')
plt.legend()
plt.show()

ValueError: cannot insert time, already exists

In [15]:
anomaly_df.to_csv('files/3SigmaAnomaly.csv')

import plotly.graph_objs as go
import plotly.offline as py
#Plot predicted and actual line graph with X=dates, Y=Outbound
actual_chart = go.Scatter(x=no_anomaly_df['SerialNo'], y=no_anomaly_df['cnt'], name= 'Data without Anomalies')
predict_chart = go.Scatter(x=anomaly_df['SerialNo'], y=anomaly_df['cnt'], name= 'Anomalies', mode='markers')
orig_chart = go.Scatter(x=df['SerialNo'], y=df['cnt'], name= 'Original Data')
py.plot([actual_chart, predict_chart, orig_chart])

NameError: name 'anomaly_df' is not defined